In [1]:
import pickle
import numpy as np
import plotly.graph_objects as go
import models.config as config

In [2]:
with open("./pred.bin", "rb") as f:
    pred = pickle.load(f)

In [3]:
config.label_dict

['IT 経済',
 'グルメ',
 'スポーツ',
 'ファッション・ビューティ',
 'ライフスタイル',
 'ライフ総合',
 '国内',
 '恋愛',
 '海外',
 '芸能',
 '車']

In [4]:
pred

<tf.Tensor: shape=(1, 11), dtype=float32, numpy=
array([[ -7.8992033, -11.692937 , -10.921753 , -12.4463625, -27.208288 ,
        -10.003849 ,  -7.5402308, -16.783327 ,  13.367447 , -11.274577 ,
        -18.618547 ]], dtype=float32)>

In [5]:
prob = nn.softmax(pred).numpy()[0]

In [6]:
prob.tolist()

[5.807787584188873e-10,
 1.3074166749227345e-11,
 2.8270568178112576e-11,
 6.154679196845869e-12,
 2.3888082735469882e-18,
 7.07905262187225e-11,
 8.315940713821135e-10,
 8.047948700691532e-14,
 1.0,
 1.9865730588319863e-11,
 1.2842783850004324e-14]

In [7]:
pred[0].numpy()

array([ -7.8992033, -11.692937 , -10.921753 , -12.4463625, -27.208288 ,
       -10.003849 ,  -7.5402308, -16.783327 ,  13.367447 , -11.274577 ,
       -18.618547 ], dtype=float32)

In [9]:
fig = go.Figure(data=[go.Pie(labels=config.label_dict, values=pred[0].numpy())])
fig.show()

In [11]:
fig.write_image("./static/images/test.png")

In [37]:
fig.show()

In [2]:
with open("../scraping/data/datasets.bin", 'rb') as f:
    datasets = pickle.load(f)

In [3]:
datasets[0]

array(['お笑いトリオ・安田大サーカスのクロちゃん(44)の所属事務所・松竹芸能は19日、クロちゃんが新型コロナウイルスに感染したことを報告した。【写真】1回目のワクチン接種後には…体温計の写真とともに経過を報告した団長安田サイトでは「7月18日夜の仕事前、体調に違和感と悪寒を感じたため、体温計測したところ37.5度あり、発熱外来を受診、新型コロナウイルスPCR検査を受けた結果、同日深夜に陽性と診断されました」と説明。「今後は保健所の指示に従い適切に対処してまいります」とした。今月15日には、団長安田(47)の新型コロナウイルス感染を公表。「先日感染をご報告させていただきました、団長安田とは直近での接触はなく、保健所からも濃厚接触者には認定されていない為、別ルートでの感染と考えられます」と伝え「この度は仕事関係者・共演者の皆様、いつも応援して下さっている皆様に多大なるご迷惑とご心配をお掛けしていることを、心よりお詫び申し上げます」と記した。最後は「弊社は引き続き、行政機関、医療機関の指導のもと、体調管理の徹底、所属タレントおよび社員、関係各位への感染防止を優先し、新型コロナウイルスの感染予防、拡大防止対策を徹底してまいります」とコメントしている。',
       '安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられます」'], dtype=object)

In [4]:
bert_tokenizer, keras_tokenizer = utils.load_tokenizer()
tf_tranformer = load_transformer()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [5]:
import pickle
import re
import mojimoji

import numpy as np

from models.config import *
from models.transformer import Transformer, create_masks

import tensorflow.nn as nn
from tensorflow import keras
from tensorflow.keras import preprocessing as pp

from transformers import BertJapaneseTokenizer

In [8]:
def generate_title(text, models, beam_width = 3):
    bt, kt, model = models
    decoder_sentence = "[BOS]"
    input_sentence = " ".join(bt.tokenize(text))
    encoder_sentence_ = kt.texts_to_sequences([input_sentence])
    encoder_sentence_ = pp.sequence.pad_sequences(encoder_sentence_, maxlen=ENC_SEQ_LEN, padding="post", truncating="post")
    for i in range(50):
        decoder_sentence_ = kt.texts_to_sequences([decoder_sentence])
        decoder_sentence_ = pp.sequence.pad_sequences(decoder_sentence_, maxlen=DEC_SEQ_LEN, padding="post", truncating="post")
        pred = model((encoder_sentence_, decoder_sentence_))
        sampled_tkn_idx = np.argmax(pred[0,i,:])
        sampled_tkn = kt.index_word[sampled_tkn_idx]
        decoder_sentence =  decoder_sentence + " " + sampled_tkn
        if sampled_tkn == "[EOS]":
            break
    return "".join(re.sub("#", "", decoder_sentence).split(" ")[1:-1])

In [9]:
generate_title(datasets[0][0], [bert_tokenizer, keras_tokenizer, tf_tranformer])

'安田大サーカス・クロちゃん、新型コロナ感染団長安田とは「別ルートでの感染と考えられない」'

# ここからビームサーチ

In [28]:
text = datasets[0][0]

idx = 0
beam_width = 3
pre_log_prob = np.array([[0.]]*beam_width)
decoder_sentence = ["[BOS]"] * beam_width

In [29]:
input_sentence = [" ".join(bert_tokenizer.tokenize(text))] * beam_width
encoder_sentence_ = keras_tokenizer.texts_to_sequences(input_sentence)
encoder_sentence_ = pp.sequence.pad_sequences(encoder_sentence_, maxlen=ENC_SEQ_LEN, padding="post", truncating="post")

In [30]:
# ここから繰り返し

In [83]:
decoder_sentence_ = keras_tokenizer.texts_to_sequences(decoder_sentence)
decoder_sentence_ = pp.sequence.pad_sequences(decoder_sentence_, maxlen=DEC_SEQ_LEN, padding="post", truncating="post")

In [84]:
decoder_sentence_, decoder_sentence_.shape

(array([[   69, 10507,   121,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [   69,   598,   177,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [   69,   177,  1503,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,   

In [85]:
encoder_sentence_.shape, decoder_sentence_.shape

((3, 1400), (3, 50))

In [86]:
pred = tf_tranformer((encoder_sentence_, decoder_sentence_))
pred.shape

TensorShape([3, 50, 28363])

In [87]:
log_prob_pred = np.log(nn.softmax(pred, axis=2))

In [88]:
log_prob_pred.shape, log_prob_pred

((3, 50, 28363),
 array([[[-39.567787 , -17.069489 , -16.214853 , ..., -37.158997 ,
          -40.21804  , -31.589146 ],
         [-36.435406 ,  -9.215334 , -10.416154 , ..., -34.65623  ,
          -35.44389  , -30.687515 ],
         [-39.18754  , -16.806036 , -14.252772 , ..., -34.777824 ,
          -43.412247 , -34.307888 ],
         ...,
         [-30.912405 , -11.96753  ,  -9.817523 , ..., -22.729021 ,
          -33.868515 , -25.0241   ],
         [-29.220476 , -13.061031 , -11.547819 , ..., -23.225607 ,
          -31.188604 , -26.660381 ],
         [-28.877745 ,  -9.493163 ,  -9.201644 , ..., -21.445559 ,
          -31.768745 , -22.333199 ]],
 
        [[-37.57273  , -18.11583  , -14.9006195, ..., -37.140236 ,
          -37.492783 , -32.667576 ],
         [-34.931744 , -15.42896  , -14.452543 , ..., -31.06536  ,
          -39.687    , -31.245728 ],
         [-28.572104 ,  -9.307538 , -10.547096 , ..., -27.821558 ,
          -35.897892 , -18.747347 ],
         ...,
         [-25.31

In [89]:
log_prob_pred = log_prob_pred[:,idx,:]
log_prob_pred.shape

(3, 28363)

In [90]:
pre_log_prob.shape

(3, 1)

In [91]:
log_prob_pred = pre_log_prob + log_prob_pred

In [92]:
log_prob_pred.shape

(3, 28363)

In [93]:
# 最初の生成だけ
if idx == 0:
    log_prob_pred = log_prob_pred[idx:idx+1]
else:
    pass

In [94]:
max_log_prob_idx_with_beam = np.argsort(-log_prob_pred.reshape(-1))[:beam_width]
max_log_prob_idx_with_beam

array([20391,  3181, 10180])

In [95]:
max_log_prob_idx_with_beam = [divmod(idx, 28363) for idx in max_log_prob_idx_with_beam]
max_log_prob_idx_with_beam

[(0, 20391), (0, 3181), (0, 10180)]

In [96]:
log_prob = [log_prob_pred[beam_idx, vocab_idx] for beam_idx, vocab_idx in max_log_prob_idx_with_beam]
log_prob

[-0.021259089757222682, -8.28830946306698, -8.643027671379969]

In [97]:
pre_log_prob += np.array(log_prob)[:,np.newaxis]
pre_log_prob

array([[ -0.04181018],
       [-20.9342675 ],
       [-23.42600725]])

In [98]:
add_words = [keras_tokenizer.index_word[vocab_idx] for _, vocab_idx in max_log_prob_idx_with_beam]
add_words

['サーカス', '##瀬', '健二']

In [99]:
decoder_sentence = [t1 + " " + t2 for t1, t2 in zip(decoder_sentence, add_words)]
decoder_sentence

['[BOS] 安田 大 サーカス', '[BOS] 23 コロナ ##瀬', '[BOS] コロナ ホテル 健二']

In [100]:
idx += 1
idx

3